In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from ultralytics import YOLO
import os
import json
import time
import matplotlib.pyplot as plt
import math
import random

In [ ]:
DIRECTORY = 'data/SoccerNetGS/gamestate-2024/valid/'


def load_data(dir: str = None) -> list:
    data = list()
    print('Wczytywanie danych...')
    for folder_name in tqdm(os.listdir(DIRECTORY)):
        folder_full_path = os.path.join(DIRECTORY, folder_name)
        if os.path.isdir(folder_full_path):
            with open(os.path.join(DIRECTORY, folder_name, 'Labels-GameState.json')) as f:
                d = json.load(f)
                data.append(d['annotations'])
    return data


def parse_data_to_df(data: list) -> pd.DataFrame:
    start = time.time()
    print('Tworzenie DataFrame...')
    flat_dict = dict()
    for annotations in tqdm(data):
        data_image = filter(lambda x: x['supercategory'] == 'object' 
                                      and x['bbox_pitch'] is not None, annotations)
        for img in data_image:
            for key in img:
                if isinstance(img[key], dict):
                    for key2 in img[key]:
                        if key + '_' + key2 not in flat_dict:
                            flat_dict[key + '_' + key2] = []
                        flat_dict[key + '_' + key2].append(img[key][key2])
                else:
                    if key not in flat_dict:
                        flat_dict[key] = []
                    flat_dict[key].append(img[key])
    df = pd.DataFrame(flat_dict, index=flat_dict['id'])
    df = df.drop(columns=['id'])
    return df

In [ ]:
def flatten_data(data: list[list[dict]]) -> list:
    flat_list = [y for x in data for y in x]
    return flat_list

In [ ]:
def get_pitch_data_df(data: list) -> pd.DataFrame:
    pitch_data_dict = {'id': [], 'image_id': [], 'line_type': [], 'x': [], 'y': []}
    allowed_lines = ['Circle central', 'Big rect. left bottom', 'Big rect. left main', 'Big rect. left top', 'Small rect. left bottom', 'Small rect. left main', 'Small rect. left top', 'Big rect. right bottom', 'Big rect. right main', 'Big rect. right top', 'Small rect. right bottom', 'Small rect. right main', 'Small rect. right top', 'Middle line', 'Side line bottom', 'Side line left', 'Side line right', 'Side line top']
    data = filter(lambda x: x['supercategory'] == 'pitch', data)
    for anotation in tqdm(data):
        lines = anotation['lines']
        lines = {k : v for k, v in lines.items() if k in allowed_lines}
        for line in lines:
            for data_point in anotation['lines'][line]:
                pitch_data_dict['id'].append(int(anotation['id']))
                pitch_data_dict['image_id'].append(int(anotation['image_id']))
                pitch_data_dict['line_type'].append(line)
                pitch_data_dict['x'].append(data_point['x'])
                pitch_data_dict['y'].append(data_point['y'])
    pitch_df = pd.DataFrame(pitch_data_dict)
    return pitch_df

In [ ]:
def denormalize_coordinates(x: float, y: float, img_width: int, img_height: int) -> tuple[int, int]:
    return int(x * img_width), int(y * img_height)


def draw_points_on_image(image: np.ndarray, coords: pd.DataFrame) -> np.ndarray:
    image_height, image_width, _ = image.shape

    colors = {
        # 3 punktowce
        'Circle central': (0, 255, 255),
        'Circle left': (255, 255, 255),
        'Circle right': (140, 159, 191),
        # lewe pole karne
        'Big rect. left bottom': (0,0,0),
        'Big rect. left main': (0, 255, 0),
        'Big rect. left top': (255, 0, 0),
        'Small rect. left bottom': (64, 255, 128),
        'Small rect. left main': (192, 255, 32),
        'Small rect. left top': (32, 192, 255),
        #prawe pole karne        
        'Big rect. right bottom': (0, 0, 255),
        'Big rect. right main': (255, 255, 0),
        'Big rect. right top': (255, 0, 255),
        'Small rect. right bottom': (255, 32, 192),
        'Small rect. right main': (32, 255, 192),
        'Small rect. right top': (192, 32, 255),
        #linie
        'Middle line': (255, 64, 128),
        'Side line bottom': (128, 255, 64),
        'Side line left': (64, 128, 255),
        'Side line right': (128, 64, 255),
        'Side line top': (255, 128, 64)
    }
    for i, row in coords.iterrows():
        color = colors[row['line_type']] if row['line_type'] in colors.keys() else (0, 0, 0)
        x_pixel, y_pixel = denormalize_coordinates(row['x'], row['y'], image_width, image_height)
        cv2.circle(image, (x_pixel, y_pixel), radius=5, color=color, thickness=-1)

    scale_factor = min(1920 / image_width, 1080 / image_height)
    if scale_factor < 1:
        image = cv2.resize(image, (int(image_width * scale_factor), int(image_height * scale_factor)))

    return image

In [ ]:
line_to_yolo_class_mapping = {
    'Circle central': 0,
    # lewe pole karne
    'Big rect. left bottom': 1,
    'Big rect. left main': 2,
    'Big rect. left top': 3,
    'Small rect. left bottom': 4,
    'Small rect. left main': 5,
    'Small rect. left top': 6,
    #prawe pole karne        
    'Big rect. right bottom': 7,
    'Big rect. right main': 8,
    'Big rect. right top': 9,
    'Small rect. right bottom': 10,
    'Small rect. right main': 11,
    'Small rect. right top': 12,
    #linie
    'Middle line': 13,
    'Side line bottom': 14,
    'Side line left': 15,
    'Side line right': 16,
    'Side line top': 17
}

In [ ]:
line_to_yolo_class_mapping.keys()

In [ ]:
def get_yolo_file_for_pitch(df: pd.DataFrame, split: str) -> None:
    file_contents = ['' for _ in range(df.image_id.nunique())]
    df = df[df['line_type'].isin(line_to_yolo_class_mapping.keys())]
    for i, row in tqdm(df.iterrows()):
        line_type: str = row['line_type']
        x: float = row['x']
        y: float = row['y']
        category: int = line_to_yolo_class_mapping[line_type]
        file_contents[int(str(row['image_id'])[-3:]) - 1] += f'{category} {x} {y}\n'
    for i, file in tqdm(enumerate(file_contents)):
        with open(f'datasets/football_lines/{split}/labels/{i + 1 :06d}.txt', 'w') as f:
            f.write(file)

In [ ]:
data: list = load_data()

In [ ]:
flatten_data_list: list[dict] = flatten_data(data)

In [ ]:
pitch_df: pd.DataFrame = get_pitch_data_df(flatten_data_list)

In [ ]:
filtered_data = list(filter(lambda x: x['supercategory'] == 'pitch', flatten_data_list))

In [ ]:
filtered_data[1]

In [ ]:
pitch_df

In [ ]:
pitch_df

In [ ]:
pitch_df_one_clip = pitch_df[pitch_df['image_id'].astype(str).str.startswith('20210')]

In [ ]:
pitch_df_one_clip

In [ ]:
def get_line_pattern(x1: float, y1: float, x2: float, y2: float) -> tuple[float, float]:
    # y = ax + b
    a = (y1 - y2) / (x1 - x2)
    b = y2 - a * x2
    return a, b

In [ ]:
def get_circle_pattern(x1: float, y1: float, x2: float, y2: float, x3: float, y3: float) -> tuple[float, float, float]:
    A = np.array([
        [x1, x2, 1],
        [x2, y2, 1],
        [x3, y3, 1]
    ])
    
    B = np.array([
        [-(x1**2 + y1**2)],
        [-(x2**2 + y2**2)],
        [-(x3**2 + y3**2)]
    ])
    
    d, e, f = np.linalg.solve(A, B).flatten()
    
    a = -d / 2
    b = -e / 2
    r = np.sqrt(pow(a, 2) + pow(b, 2) - f)
    
    return float(a), float(b), float(r)

In [ ]:
def get_ellipse_pattern(points: list) -> tuple:
    A = []
    B = []
    for (x, y) in points:
        A.append([x**2, x*y, y**2, x, y])
        B.append(1)
        
    A = np.array(A)
    B = np.array(B)

    ellipse_params = np.linalg.solve(A, B)
    
    return tuple(ellipse_params.tolist())
    

In [ ]:
def get_ellipse_center(func: tuple) -> tuple:
    A, B, C, D, E = func
    x = (B * E - 2 * C * D) / (4 * A * C - B**2)
    y = (B * D - 2 * A * E) / (4 * A * C - B**2)
    return x, y

In [ ]:
def get_ellipse_axes(func: tuple) -> tuple:
    A, B, C, D, E = func
    M = np.array([[A, B / 2], [B / 2, C]])

    eigenvalues = np.linalg.eigvals(M)
    lambda1, lambda2 = eigenvalues[0], eigenvalues[1]

    delta = (B**2 - 4 * A * C)
    if delta == 0:
        raise ValueError("Nie jest to równanie elipsy lub opis jest niewłaściwy.")

    a = math.sqrt(-delta / lambda1)
    b = math.sqrt(-delta / lambda2)
    return a, b

In [ ]:
lista = [1, 2, 3]
lista[:3]

In [ ]:
def get_func_pattern(df: pd.DataFrame) -> dict[str, list[tuple]]:
    func_patterns = {'line': [], 'ellipse': []}
    df_list: list[pd.DataFrame] = [image_id for _, image_id in df.groupby('image_id')]
    for df_of_image in tqdm(df_list):
        image_id: int = df_of_image.iloc[0]['image_id']
        points_dict: dict[str, list] = {key: [] for key in df_of_image['line_type'].unique()}
        for i, row in df_of_image.iterrows():
            points_dict[row['line_type']].append((row['x'], row['y']))
        for line in points_dict.keys():
            if 'Circle' in line and len(points_dict[line]) >= 5:
                points: list[tuple[float, float]] = random.sample(points_dict[line], 5)
                points = [(x * 1920, y * 1080) for x, y in points]
                func_pattern = get_ellipse_pattern(points)
                func_patterns['ellipse'].append((int(image_id), func_pattern))
            else:
                points = random.sample(points_dict[line], 2)
                flatten_points = []
                for x, y in points:
                    flatten_points.append(x * 1920)
                    flatten_points.append(y * 1080)
                func_pattern = get_line_pattern(*flatten_points)
                func_patterns['line'].append((int(image_id), func_pattern))
    return func_patterns

In [ ]:
lines = get_func_pattern(pitch_df_one_clip)

In [ ]:
def draw_line_on_image(image: np.array, func: tuple, give_pattern: bool = False) -> np.array:
    a: float = func[0]
    b: float = func[1]
    if give_pattern:
        print(f'f(x) = {a}x + {b}')
        print(image.shape)
    x = np.linspace(0, image.shape[1], 1000)
    y = a * x + b
    filtered_x = list()
    filtered_y = list()
    for x_val, y_val in zip(x, y):
        if 0 <= y_val <= image.shape[0]:
            filtered_x.append(x_val)
            filtered_y.append(y_val)
    x = filtered_x
    y = filtered_y
    cv2.line(image, (int(x[0]), int(y[0])), (int(x[-1]), int(y[-1])), color=(0,0,0), thickness=2)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

In [ ]:
def draw_circle_on_image(image: np.array, func: tuple, give_pattern: bool = False):
    a: float = int(func[0])
    b: float = int(func[1])
    r: float = int(func[2])
    
    cv2.circle(image, (a, b), r, color=(0,0,0), thickness=2)
    if give_pattern:
        print(f'(x - {a})^2 + (y - {b})^2 = {r}^2')
    
    cv2.imshow("Koło", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def draw_ellipse_on_image(image: np.array, func: tuple, give_pattern: bool = False) -> np.array:
    A, B, C, D, E = func
    if give_pattern:
        print(f'{A:.20f}x^2 + {B:.20f}xy + {C:.20f}y^2 + {D:.20f}x + {E:.20f}y = 1')
    middle_x, middle_y = get_ellipse_center(func)
    a, b = get_ellipse_axes(func)
    cv2.ellipse(image, (int(middle_x), int(middle_y)), (int(a), int(b)), 0, 0, 360, color=(0,0,0), thickness=2)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image_rgb

In [ ]:
def draw_all_lines(image: np.array, lines: list) -> np.array:
    for line in lines:
        if len(line) == 2:
            image = draw_line_on_image(image, line)
        # else:
        #     image = draw_ellipse_on_image(image, line)
    return image

In [ ]:
pitch_df_one_image = pitch_df_one_clip[pitch_df_one_clip['image_id'] == 2021000001]
pitch_df_one_image_circle_coords = pitch_df_one_image[pitch_df_one_image['line_type'].str.contains('Circle')]
ellipse_points = [(row['x']*1920, row['y']*1080) for i, row in pitch_df_one_image_circle_coords.iterrows()]
ellipse_points

In [ ]:
ellipse_points_sample = random.sample(ellipse_points, 5)
A, B, C, D, E = get_ellipse_pattern(ellipse_points_sample)

In [ ]:
center = get_ellipse_center((A, B, C, D, E))

In [ ]:
def is_point_on_ellipse(point: tuple):
    result = point[0]**2 * A + point[0] * point[1] * B + point[1]**2 * C + point[0] * D + point[1] * E - 1
    if result < 0.005:
        return True
    return False

In [ ]:
draw_ellipse_on_image(cv2.imread('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg'), (A, B, C, D, E))

In [ ]:
lines_one_image = get_func_pattern(pitch_df_one_image)
lines_one_image

In [ ]:
path = 'data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg'
image = cv2.imread(path)
draw_circle_on_image(image, lines['circle'][0][1])

In [ ]:
path = 'data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg'
draw_line_on_image(path, lines_one_image['line'][1][1])

In [ ]:
img = cv2.imread(path)
edited = draw_points_on_image(img, pitch_df_one_clip[pitch_df_one_clip['image_id'] == 1060000001])

In [ ]:
cv2.imshow("img with points", edited)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def get_linear_crossing_points(f1: tuple, f2: tuple, im_width: int = 1920, im_height: int = 1080) -> tuple | None:
    if len(f1) != len(f2) != 2:
        raise ValueError('Functions are not linear')
    x: float = (f2[1] - f1[1]) / (f1[0] - f2[0])
    y: float = f1[0] * x + f1[1]
    if x < 0 or x> im_width or y < 0 or y > im_height:
        return None
    return x, y

In [ ]:
def get_linear_with_ellipse_crossing(f1: tuple, f2: tuple, im_width: int = 1920, im_height: int = 1080) -> list | None:
    if len(f1) != 2 or len(f2) != 5:
        raise ValueError('Not right function type')
    # Ellipse like Ax^2 + Bxy + Cy^2 + Dx + Ey = 1
    A: float = f2[0]
    B: float = f2[1]
    C: float = f2[2]
    D: float = f2[3]
    E: float = f2[4]
    # Linear like y = a1 * x + b1
    a1: float = f1[0]
    b1: float = f1[1]
    
    a: float = A + B * a1 + C * pow(a1, 2)
    b: float = B * b1 + 2 * C * b1 * a1 + D + E * a1
    c: float = C * pow(b1, 2) + E * b1 - 1
        
    delta: float = pow(b, 2) - 4 * a * c
    
    if delta < 0:
        return None
    
    delta_sqrt: float = math.sqrt(delta)
    x1 : float = (-b + delta_sqrt) / (2 * a)
    y1: float = a1 * x1 + b1
    x2 : float = (-b - delta_sqrt) / (2 * a)
    y2 : float = a1 * x2 + b1
    
    x1 = x1 if 0 <= x1 <= im_width else None
    y1 = y1 if 0 <= y1 <= im_height else None
    x2 = x2 if 0 <= x2 <= im_width else None
    y2 = y2 if 0 <= y2 <= im_height else None

    to_return = list()

    if x1 is not None and y1 is not None:
        to_return.append((x1, y1))
    
    if x2 is not None and y2 is not None:
        to_return.append((x2, y2))
    
    if len(to_return) > 0:
        return to_return
    
    return None

In [ ]:
points = get_linear_with_ellipse_crossing((74.71961698993614, -75573.49002185368), (A, B, C, D, E))

In [ ]:
linear_points = get_linear_crossing_points(lines_one_image['line'][1][1], lines['line'][0][1])

In [ ]:
img = cv2.imread('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg')
points = points + [linear_points] + [center]
for point in points:
    cv2.circle(img, (int(point[0]), int(point[1])), radius=5, color=(0, 0, 255), thickness=-1)
cv2.imshow("img with points", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
get_linear_crossing_points((4,17), (-13, 3))

In [ ]:
# {'line': [(2021000001, (95.44767670414602, -96609.38215087929)),
#   (2021000001, (-0.014856099787039952, 270.584891142075))],
#  'circle': [(2021000001,
#    array([-1.2458e-07, -4.4927e-09, -3.3171e-06,  0.00025738,   0.0034819]))]}
lines_for_draw = list()
for key in lines_one_image.keys():
    for item in lines_one_image[key]:
        lines_for_draw.append(item[1])
lines_for_draw


In [ ]:
image_result = draw_all_lines(cv2.imread('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000001.jpg'), lines_for_draw)

In [ ]:
image_rgb = cv2.cvtColor(image_result, cv2.COLOR_BGR2RGB)


In [ ]:
test_img = cv2.imread('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000414.jpg')

In [ ]:
lines_1 = get_func_pattern(pitch_df_one_clip[pitch_df_one_clip['image_id'] == 2021000001])
lines_1 

In [ ]:
lines_414 = get_func_pattern(pitch_df_one_clip[pitch_df_one_clip['image_id'] == 2021000414])
lines_414

In [ ]:
draw_ellipse_on_image(test_img, lines_414['ellipse'][0][1], True)

In [ ]:
test_lines = [x[1] for x in lines_414['line']]
all_lines = draw_all_lines(test_img, test_lines)

In [ ]:
scaled_img = cv2.resize(all_lines, (920, 540))
cv2.imshow("img with lines", scaled_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from keypoint import Keypoint

In [ ]:
keypoint = Keypoint()
funcs = keypoint.get_func_pattern(pitch_df_one_clip)

In [ ]:
funcs[np.int64(2021000001)]

In [ ]:
get_linear_crossing_points((np.float64(74.71961698993614), np.float64(-75573.49002185368)),(np.float64(-0.0204907665064508), np.float64(278.61844396660763)))
get_linear_crossing_points((np.float64(74.71961698993614), np.float64(-75573.49002185368)),(np.float64(-0.0204907665064508), np.float64(278.61844396660763)))

In [ ]:
def get_crossing_matrix():
    crossing_matrix = np.zeros((18, 18))
    crossing_matrix[0, 13] = 1
    crossing_matrix[13, 0] = 1
    crossing_matrix[13, 14] = 1
    crossing_matrix[14, 13] = 1
    crossing_matrix[13, 17] = 1
    crossing_matrix[17, 13] = 1

    crossing_matrix[1, 15] = 1
    crossing_matrix[15, 1] = 1
    crossing_matrix[1, 2] = 1
    crossing_matrix[2, 1] = 1
    crossing_matrix[3, 2] = 1
    crossing_matrix[2, 3] = 1
    crossing_matrix[3, 15] = 1
    crossing_matrix[15, 3] = 1

    crossing_matrix[4, 15] = 1
    crossing_matrix[15, 4] = 1
    crossing_matrix[4, 5] = 1
    crossing_matrix[5, 4] = 1
    crossing_matrix[6, 5] = 1
    crossing_matrix[5, 6] = 1
    crossing_matrix[6, 15] = 1
    crossing_matrix[15, 6] = 1

    crossing_matrix[14, 15] = 1
    crossing_matrix[15, 14] = 1
    crossing_matrix[15, 17] = 1
    crossing_matrix[17, 15] = 1

    crossing_matrix[7, 16] = 1
    crossing_matrix[16, 7] = 1
    crossing_matrix[7, 8] = 1
    crossing_matrix[8, 7] = 1
    crossing_matrix[9, 8] = 1
    crossing_matrix[8, 9] = 1
    crossing_matrix[9, 16] = 1
    crossing_matrix[16, 9] = 1

    crossing_matrix[10, 16] = 1
    crossing_matrix[16, 10] = 1
    crossing_matrix[10, 11] = 1
    crossing_matrix[11, 10] = 1
    crossing_matrix[12, 11] = 1
    crossing_matrix[11, 12] = 1
    crossing_matrix[12, 16] = 1
    crossing_matrix[16, 12] = 1

    crossing_matrix[14, 16] = 1
    crossing_matrix[16, 14] = 1
    crossing_matrix[16, 17] = 1
    crossing_matrix[17, 16] = 1
    return crossing_matrix

In [ ]:
def get_all_keypoints_by_id(img_id: int) -> list:
    crossing_matrix = get_crossing_matrix()
    lines = funcs[img_id]
    matches = list()
    keypoints = list()
    print(lines.keys())
    for line in lines.keys():
        possible_crossings = np.where(crossing_matrix[line] == 1)
        possible_crossings = possible_crossings[0].tolist()
        print(line)
        print(possible_crossings)
        for crossing in possible_crossings:
            if crossing in lines.keys():
                if line < crossing:
                    matches.append((line, crossing))
                else:
                    matches.append((crossing, line))
    matches = set(matches)
    for match in matches:
        if match[0] == 0:
            keypoints += get_linear_with_ellipse_crossing(lines[match[1]], lines[0])
        else:
            keypoints.append(get_linear_crossing_points(lines[match[0]], lines[match[1]]))

    return keypoints

In [ ]:
keypoints = get_all_keypoints_by_id(np.int64(2021000750))

In [ ]:
keypoints

In [ ]:
np.ndarray([ 0, 14, 15])


In [ ]:
image = cv2.imread('data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1/000750.jpg', cv2.IMREAD_COLOR)
for point in keypoints:
    if point is not None:
        print(point[0])
        print(point[1])
        cv2.circle(image, (int(point[0]), int(point[1])), radius=5, color=(0, 0, 255), thickness=-1)
image = cv2.resize(image, (920, 540))
cv2.imshow("img with points", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
strowanie = '0 0.5 0.59921875 1 0.49921875 0.17810874999999998 0.36350546296296293 2 0.12096354166666665 0.3944155555555555 2 0.05939333333333333 0.4264648148148148 2 0.023809531249999998 0.6509474074074073 0 0.019841249999999998 0.6998087962962962 0 0.019841249999999998 0.8001728703703703 0 0.10846963541666665 0.4283962962962963 2 0.08994708333333332 0.6509474074074073 0 0.11180828125 0.4630316666666666 2 0.26169635416666664 0.39601777777777775 2 0.21068723958333332 0.43467879629629624 2 0.12624916666666666 0.4994755555555555 2 0.011346458333333333 0.5872359259259259 2 0.6094970833333333 0.3660373148148148 2 0.6150390625 0.4345051851851852 2 0.62099609375 0.5273582407407408 2 0.6400329687499999 0.8180375 2 0.9859110416666667 0.4053819444444445 2 0.70634921875 0.5505833333333333 0 0.7037036979166666 0.6496268518518518 0 0.70634921875 0.7024499999999999 0 0.7989418229166667 0.6007653703703704 0 0.9047619270833334 0.5479422222222222 0 0.9047619270833334 0.6509474074074073 0 0.98015875 0.39739620370370365 0 0.98015875 0.49908074074074077 0 0.98015875 0.5479422222222222 0 0.9761904687499999 0.6509474074074073 0 0.98015875 0.6998087962962962 0 0.9761904687499999 0.8014934259259258 0 0.4973246354166666 0.4740993518518518 2 0.7400675520833333 0.4766275925925926 2'
strowanie.split()